In [1]:
import json
from StringIO import StringIO
import pandas as pd
import openpyxl

In [2]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind 
import csv

C:\Users\sanka\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext(master="local[3]") 
from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [4]:
from pyspark.sql import Row
from pyspark.sql.types import *

In [5]:
listings = pd.read_csv('inside_airbnb/listings.csv')

In [6]:
special_events = pd.read_csv('Datasources/sdcounty_datasets/special_events_list_datasd.csv')

In [7]:
schema = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)])

In [8]:
schema_two = StructType([StructField("event_id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)])

In [9]:
spql_listings_info = sqlContext.createDataFrame(listings[['id', 'longitude', 'latitude']], schema)

In [10]:
spql_special_events_info = sqlContext.createDataFrame(special_events[['event_id', 'longitude', 'latitude']].dropna(), schema_two)

In [11]:
spql_listings_info_rdd = spql_listings_info.rdd.map(lambda row:(row.id, row.longitude,row.latitude))

In [12]:
spql_special_events_info_rdd = spql_special_events_info.rdd.map(lambda row:(row.event_id, row.longitude,row.latitude))

In [13]:
spql_listings_and_events_info_rdd = spql_listings_info_rdd.cartesian(spql_special_events_info_rdd)

In [14]:
spql_listings_and_events_info_rdd = spql_listings_and_events_info_rdd.map(lambda x: (x[0][0], x[0][1], x[0][2], x[1][0], x[1][1], x[1][2]))

In [15]:
def distance_calculation(x):
        p_one = float(x[2])
        q_one = float(x[1])
        p_two = float(x[5])
        q_two = float(x[4])
        lon_diff = (q_one - q_two)*np.pi/180
        lat_diff = (p_one - p_two)*np.pi/180
        a = np.sin(lat_diff/2)**2 + np.cos(p_one*np.pi/180)*np.cos(p_two*np.pi/180)*(np.sin(lon_diff/2)**2)
        c = np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        d = 6371.00*float(c)
        return tuple(list(x) + [d])

In [16]:
spql_listings_and_events_info_rdd = spql_listings_and_events_info_rdd.map(distance_calculation)

In [17]:
schema_three = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)] + [StructField("event_id", IntegerType(), True),
                    StructField("event_longitude", DoubleType(), True),
                    StructField("event_latitude", DoubleType(), True), StructField("distance", DoubleType(), True)])

In [18]:
spql_listings_and_events_info_df = sqlContext.createDataFrame(spql_listings_and_events_info_rdd, schema_three)

In [19]:
spql_listings_and_events_info_rdd = spql_listings_and_events_info_df.rdd.map(lambda row: (row.id, row.longitude, row.latitude, row.event_id, row.event_longitude, row.event_latitude, row.distance))

In [20]:
closest_event_distance = spql_listings_and_events_info_rdd.map(lambda x: (x[0], x[-1])).reduceByKey(lambda x, y : min(x, y))

In [ ]:
closest_event_distance = spql_listings_and_events_info_rdd.cartesian(closest_event_distance).filter(lambda x: x[0][-1] == x[1][-1]).map(lambda x: (x[0][0], x[0][-1], x[0][3]))

In [21]:
close_events_count_one = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 1).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_events_count_three = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 3).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_events_count_five = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 5).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_events_count_ten = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 10).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [22]:
close_parks_count_sixteen = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 16).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [39]:
count = 0

In [40]:
import datetime
today = datetime.date.today()
count+=1
filename = 'Datasources/listings_events/listings_events_' + str(today) + '_V' + str(count) + '.xlsx'

In [41]:
print filename

Datasources/listings_events/listings_events_2018-05-20_V1.xlsx


In [ ]:
df1 = close_events_count_one.toDF()

In [ ]:
df2 = close_events_count_three.toDF()

In [ ]:
df3 = close_events_count_five.toDF()

In [ ]:
df4 = close_events_count_ten.toDF()

In [ ]:
df5 = close_parks_count_sixteen.toDF()

In [ ]:
df6 = closest_park_distance.toDF()

In [ ]:
df1 = df1.toPandas()

In [ ]:
df1 = df1.rename(columns={'_1':'listing_id', '_2':'count_1km'})

In [ ]:
df2 = df2.toPandas()

In [ ]:
df2 = df2.rename(columns={'_1':'listing_id', '_2':'count_3km'})

In [ ]:
df3 = df3.toPandas()

In [ ]:
df3 = df3.rename(columns={'_1':'listing_id', '_2':'count_5km'})

In [ ]:
df4 = df4.toPandas()

In [ ]:
df4 = df4.rename(columns={'_1':'listing_id', '_2':'count_10km'})

In [ ]:
df5 = df5.toPandas()

In [ ]:
df5 = df5.rename(columns={'_1':'listing_id', '_2':'count_16km'})

In [ ]:
df6 = df6.toPandas()